# Exploratory Data Analysis

<span style="color:blue">Disclaimer: We began our analysis with a set of filters aimed to include all loan applications for single family homes purchased as their primary residency. After, performing the data analysis below,  we noticed that our filters included loan applications that where either withdrawn by applicant or closed due to incompleteness (variable action taken 4,5); in our subsequent analysis we decided to exclude these two possible outcomes, since these loan applications may contain incomplete or inaccurate information. 
We also discovered that our filters were limiting to purchaser type to ‘Non-applicable’, we decided to remove this filter and include all loan purchaser types, since the large majority of loans in the US are conforming loans, which are loans that comply with the guidelines on the Federal Housing Finance agency (ref: https://en.wikipedia.org/wiki/Conforming_loan,  https://en.wikipedia.org/wiki/Fannie_Mae ) and backed by organizations such as Fannie Mae and Freddie Mac and we wanted to study a representative set of home loans and avoid selection bias.</span>.




The goal of this notebook was to create a single pandas dataframe with which we could explore most of our data. The other notebooks are more complete in their exposition as they focus on more specific questions and have more defined outcomes. Furthermore, later we apply different filters to take a larger slice of the data, as we broaden the analysis, therefore the statistics are different.

In [ ]:
#import libraries and settings
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql import SparkSession

import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.disable_max_rows()
alt.themes.enable("fivethirtyeight")

from scipy.stats import ttest_ind
import statsmodels.formula.api as sm

The first task is to load the mortgage application dataset (https://ffiec.cfpb.gov/data-publication/dynamic-national-loan-level-dataset/2021). The file is too large for Pandas so we need Spark, however, for EDA I want a single Pandas dataframe.

In [ ]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('My First Spark application') \
    .getOrCreate()
sc = spark.sparkContext

df_hm = spark.read.option("header",True) \
     .csv("2021_public_lar.csv")

df_hm.createOrReplaceTempView('N_df_view')

To begin we restrict our analysis to the most 'standard' types of mortgages. Going through the schema I selected options for each feature that is not a variable of interest, aside from the columns that do not contain valuable information. This reduces the size of the dataframe as much as possible in order to help produce that single Pandas dataframe.

In [ ]:
def cut_view_red():
    return spark.sql("""\
        SELECT *
        FROM N_df_view
        WHERE derived_loan_product_type = "Conventional:First Lien" AND
        conforming_loan_limit = "C" AND
        lien_status = 1 AND
        reverse_mortgage = 2 AND
        open_end_line_of_credit = 2 AND
        negative_amortization = 2 AND
        balloon_payment = 2 AND
        total_units = 1 AND
        derived_dwelling_category = 'Single Family (1-4 Units):Site-Built' AND
        purchaser_type = 0 AND
        loan_type = 1 AND
        loan_purpose = 1 AND
        business_or_commercial_purpose = 2 AND
        occupancy_type = 1
        """)

results = cut_view_red()

Below are our initial variables of interest as described in our proposal document. I select on these to drop unwanted columns.

In [ ]:
results = results.select('county_code',
                        'census_tract',
                        'derived_ethnicity', 
                        'derived_race', 
                        'action_taken', 
                        'combined_loan_to_value_ratio',
                        'interest_rate',
                        'property_value',
                        'income',
                        'debt_to_income_ratio',
                        'rate_spread')

I can now go ahead and create a Pandas dataframe, and for the sake of memory I drop the Spark dataframe and view.

In [ ]:
DF = results.toPandas()
del df_hm
del results

## Race and Ethnicity 

We want to understand what data is contained within the columns of the dataframe that relate to Race and Ethnicity.

In [ ]:
DF['derived_race'].value_counts()

As you can see there are labels that correspond to very small proportions of the dataset as a whole. For simplicity lets drop these for now. The same goes for ethnicity.

In [ ]:
DF = DF.drop(DF[
    (DF['derived_race'] == 'Free Form Text Only') |
    (DF['derived_race'] == 'Race Not Available') |
    (DF['derived_race'] == 'Native Hawaiian or Other Pacific Islander') | 
    (DF['derived_race'] == '2 or more minority races') |
    (DF['derived_race'] == 'American Indian or Alaska Native') |
    (DF['derived_race'] == 'Joint')
].index)

In [ ]:
DF['derived_ethnicity'].value_counts()

In [ ]:
DF = DF.drop(DF[
    (DF['derived_ethnicity'] == 'Ethnicity Not Available') |
    (DF['derived_ethnicity'] == 'Free Form Text Only') |
    (DF['derived_ethnicity'] == 'Joint')
].index)

Below we overwrite all races with ethnicity 'Hispanic'.

In [ ]:
DF.loc[(DF['derived_ethnicity'] == 'Hispanic or Latino'), 'derived_race'] = 'Hispanic'

# Alternatively we can split the race 'white' into 'white' and 'Hispanic'. 
# DF.loc[(DF['derived_ethnicity'] == 'Hispanic or Latino') & 
#           (DF['derived_race'] == 'White'), 'derived_race'] = 'Hispanic'

In [ ]:
DF.head()

## Joining datasets

Our goal is to investigate the relationship between historical redlining and mortgage applications, so we will need to add in the former. Below we load the HOLC dataset and calculate the Historical Redlining score. This is explained in the HRS map notebook.

In [ ]:
holc=pd.read_csv('HOLC_2020_census_tracts.csv')
holc_rated= holc.copy(deep=True)
holc_rated['a']= ((holc_rated['area_A'])/holc_rated['area_rated'])*1
holc_rated['b']= ((holc_rated['area_B'])/holc_rated['area_rated'])*2
holc_rated['c']= ((holc_rated['area_C'])/holc_rated['area_rated'])*3
holc_rated['d']= ((holc_rated['area_D'])/holc_rated['area_rated'])*4
holc_rated['HRS']= holc_rated[['a', 'b', 'c', 'd']].sum(axis=1)
holc_rated = holc_rated[['class1','geoid20', 'HRS']]
holc_rated = holc_rated.rename(columns = {'geoid20':'census_tract'})

I use the census tract number to add the HRS to our mortgage application data, then begin to explore the first hypothesis as set out in our proposal, namely, are different groups charged different rates. Firstly there are empty and non-numeric values in those variables of interest. Also the data types are not set correctly. I deal with these below   

In [ ]:
DF = DF[pd.to_numeric(DF['census_tract'], errors='coerce').notnull()]
DF = DF[pd.to_numeric(DF['interest_rate'], errors='coerce').notnull()]
DF = DF[pd.to_numeric(DF['rate_spread'], errors='coerce').notnull()]
DF = DF[pd.to_numeric(DF['combined_loan_to_value_ratio'], errors='coerce').notnull()]

DF['census_tract'] = DF['census_tract'].astype(np.int64)
DF['interest_rate'] = DF['interest_rate'].astype(float)
DF['rate_spread'] = DF['rate_spread'].astype(float)
DF['combined_loan_to_value_ratio'] = DF['combined_loan_to_value_ratio'].astype(float)

I can now add a column for the historical redlining score to the mortgage application dataframe

In [ ]:
DF1 = pd.merge(DF, holc_rated, how='left', on='census_tract')
DF1 = DF1.dropna(subset=['HRS'])

### Mean Interest rates by Race/Ethnicity

As you can see below, there are differences in the mean interest rate paid by different groups, and also the mean rate spread, which is the difference between the interest rate and the average prime offer rate (APOR) for a comparable transaction. These differences are statistically significant. In the other notebooks we apply these calculations to a broader section of the dataset, so the results are different.

In [ ]:
DF1.groupby(['derived_race'])['interest_rate'].mean()

In [ ]:
DF1.groupby(['derived_race'])['rate_spread'].mean()

In [ ]:
ttest_ind(DF1[DF1['derived_race'] == 'Hispanic']['interest_rate'],
          DF1[DF1['derived_race'] == 'White']['interest_rate'])

In [ ]:
ttest_ind(DF1[DF1['derived_race'] == 'Hispanic']['rate_spread'],
          DF1[DF1['derived_race'] == 'White']['rate_spread'])

Below we look for correlations between the HRS and other variables of interest. As you can see there are outliers in the interest rate and combined loan to value ratio. There is also a concentration of datapoints where the HRS takes an integer value. There also appears to be a very low correlation between the HRS score and other variables of interest. This is not good news for any potential regression analysis. 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.set_title('Interest v HRS')
ax1.plot(DF1['HRS'], DF1['interest_rate'],'.')
ax2.set_title('Interest v combined_loan_to_value_ratio')
ax2.plot(DF1['combined_loan_to_value_ratio'], DF1['interest_rate'],'.')

In [ ]:
np.corrcoef(DF1['HRS'], DF1['interest_rate'])

In [ ]:
np.corrcoef(DF1['combined_loan_to_value_ratio'], DF1['interest_rate'])

## Applications by HRS and Race/Ethnicity

It would be instructive to get a broad overview of mortgage applications by HRS and race. In order to do so we created bins which correspond to the grades ungraded, A, B, C, D.

In [ ]:
AltChart1DF = DF1.copy(deep=True)
AltChart1DF = AltChart1DF[['HRS','derived_race']]

In [ ]:
conditions = [
    (AltChart1DF['HRS'] <= 1.75),
    (AltChart1DF['HRS'] > 1.76) & (AltChart1DF['HRS'] <= 2.49),
    (AltChart1DF['HRS'] > 2.5) & (AltChart1DF['HRS'] <= 3.3),
    (AltChart1DF['HRS'] > 3.3)
    ]

# create a list of the values we want to assign for each condition
values = ['HRS 1-1.75', 'HRS 1.76-2.49', 'HRS 2.50-3.3', 'HRS > 3.3']

# create a new column and use np.select to assign values to it using our lists as arguments
AltChart1DF['HRS binned'] = np.select(conditions, values)

In [ ]:
alt.Chart(AltChart1DF).mark_bar().encode(
    y=alt.Y('HRS binned:N'),
    x=alt.X('count(HRS binned):Q',
    title='Number of Applications'),
    color=alt.Color('derived_race:N',
    legend=alt.Legend(title='Race or Ethnicity')),
    order=alt.Order('derived_race',
      sort='descending'
    )
).properties(height=400, width=300,
    title='Loan Applications by HRS (binned)').configure_axis(
    grid=False
).configure_title(fontSize=14).configure(background='#FFFFFF').configure_axis(
    grid=False)


### Approval/Denial by race

Below we take a quick look at the raw numbers for mortgage approval/denial by race

In [ ]:
DF1['action_taken'].replace(['8','2'], '0', inplace=True)

In [ ]:
DF1.groupby(['derived_race','action_taken'])['action_taken'].count()